<a href="https://colab.research.google.com/github/mfareed4/COVID-19-Global-Dashboard/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install pandas plotly dash requests pycountry > /dev/null

# Import libraries
import pandas as pd
import requests
from datetime import datetime
import pycountry
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Fetch and process COVID-19 data
def fetch_covid_data():
    """Fetch and process COVID-19 data from disease.sh API"""
    url = "https://disease.sh/v3/covid-19/countries"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Transform to DataFrame
        df = pd.json_normalize(data)

        # Clean and select columns
        columns = {
            'country': 'Country',
            'cases': 'Total Cases',
            'todayCases': 'New Cases',
            'deaths': 'Total Deaths',
            'todayDeaths': 'New Deaths',
            'recovered': 'Total Recovered',
            'active': 'Active Cases',
            'critical': 'Critical Cases',
            'casesPerOneMillion': 'Cases Per Million',
            'deathsPerOneMillion': 'Deaths Per Million',
            'tests': 'Total Tests',
            'testsPerOneMillion': 'Tests Per Million',
            'population': 'Population',
            'continent': 'Continent',
            'countryInfo.lat': 'Latitude',
            'countryInfo.long': 'Longitude'
        }
        df = df.rename(columns=columns)[list(columns.values())]

        # Calculate additional metrics
        df['Recovery Rate'] = (df['Total Recovered'] / df['Total Cases'].replace(0, pd.NA)).fillna(0) * 100
        df['Fatality Rate'] = (df['Total Deaths'] / df['Total Cases'].replace(0, pd.NA)).fillna(0) * 100
        df['Test Positivity Rate'] = (df['Total Cases'] / df['Total Tests'].replace(0, pd.NA)).fillna(0) * 100

        # Add ISO alpha-3 codes for better map visualization
        def get_iso_alpha3(country_name):
            try:
                return pycountry.countries.get(name=country_name).alpha_3
            except:
                try:
                    return pycountry.countries.search_fuzzy(country_name)[0].alpha_3
                except:
                    return None

        df['ISO'] = df['Country'].apply(get_iso_alpha3)

        # Add timestamp
        df['Last Updated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

# Fetch the data
df = fetch_covid_data()

# Preprocess continent data
continent_data = df.groupby('Continent').agg({
    'Total Cases': 'sum',
    'Total Deaths': 'sum',
    'Total Recovered': 'sum',
    'Population': 'sum'
}).reset_index()
continent_data['Fatality Rate'] = (continent_data['Total Deaths'] / continent_data['Total Cases'].replace(0, pd.NA)).fillna(0) * 100

# Initialize the app
app = Dash(__name__)

# Custom CSS styles
styles = {
    'metric-box': {
        'backgroundColor': '#f8f9fa',
        'borderRadius': '5px',
        'padding': '20px',
        'margin': '10px',
        'boxShadow': '2px 2px 5px rgba(0,0,0,0.1)',
        'textAlign': 'center'
    },
    'graph-container': {
        'padding': '10px',
        'width': '48%',
        'display': 'inline-block'
    },
    'row': {
        'display': 'flex',
        'justifyContent': 'space-between',
        'marginBottom': '20px'
    },
    'footer': {
        'textAlign': 'center',
        'marginTop': '30px',
        'color': '#666',
        'fontSize': '12px'
    }
}

# App layout
app.layout = html.Div([
    html.H1("COVID-19 Global Dashboard", style={'textAlign': 'center', 'color': '#2c3e50'}),

    # Top metrics row
    html.Div([
        html.Div([
            html.H3("Total Cases Worldwide", style={'color': '#e74c3c'}),
            html.P(f"{df['Total Cases'].sum():,}", style={'fontSize': '24px', 'fontWeight': 'bold'})
        ], style=styles['metric-box']),

        html.Div([
            html.H3("Total Deaths Worldwide", style={'color': '#7f8c8d'}),
            html.P(f"{df['Total Deaths'].sum():,}", style={'fontSize': '24px', 'fontWeight': 'bold'})
        ], style=styles['metric-box']),

        html.Div([
            html.H3("Global Fatality Rate", style={'color': '#c0392b'}),
            html.P(f"{(df['Total Deaths'].sum() / df['Total Cases'].sum() * 100 if df['Total Cases'].sum() > 0 else 0):.2f}%",
                  style={'fontSize': '24px', 'fontWeight': 'bold'})
        ], style=styles['metric-box']),

        html.Div([
            html.H3("Countries Affected", style={'color': '#16a085'}),
            html.P(f"{len(df['Country'].unique())}", style={'fontSize': '24px', 'fontWeight': 'bold'})
        ], style=styles['metric-box'])
    ], style={'display': 'flex', 'justifyContent': 'space-around', 'marginBottom': '30px'}),

    # First visualization row
    html.Div([
        html.Div([
            dcc.Graph(id='cases-map'),
            dcc.Dropdown(
                id='metric-dropdown',
                options=[
                    {'label': 'Total Cases', 'value': 'Total Cases'},
                    {'label': 'Total Deaths', 'value': 'Total Deaths'},
                    {'label': 'Active Cases', 'value': 'Active Cases'},
                    {'label': 'Fatality Rate', 'value': 'Fatality Rate'},
                    {'label': 'Recovery Rate', 'value': 'Recovery Rate'},
                ],
                value='Total Cases',
                clearable=False,
                style={'width': '50%', 'margin': '0 auto'}
            )
        ], style=styles['graph-container']),

        html.Div([
            dcc.Graph(
                id='continent-bar',
                figure=px.bar(
                    continent_data,
                    x='Continent',
                    y='Total Cases',
                    title='Cases by Continent (Log Scale)',
                    color='Continent',
                    hover_data=['Total Deaths', 'Fatality Rate'],
                    log_y=True
                ).update_layout(
                    showlegend=False,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)'
                )
            )
        ], style=styles['graph-container'])
    ], style=styles['row']),

    # Second visualization row
    html.Div([
        html.Div([
            dcc.Graph(
                id='recovery-scatter',
                figure=px.scatter(
                    df[df['Total Cases'] > 0],
                    x='Total Cases',
                    y='Total Recovered',
                    size='Population',
                    color='Continent',
                    hover_name='Country',
                    log_x=True,
                    log_y=True,
                    title='Cases vs Recovered (Log Scale)',
                    trendline="lowess",
                    trendline_color_override="red"
                ).update_layout(
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)'
                )
            )
        ], style=styles['graph-container']),

        html.Div([
            dcc.Graph(
                id='fatality-pie',
                figure=px.pie(
                    continent_data,
                    names='Continent',
                    values='Total Deaths',
                    title='Death Distribution by Continent',
                    hole=0.4,
                    color_discrete_sequence=px.colors.sequential.RdBu
                ).update_layout(
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)'
                )
            )
        ], style=styles['graph-container'])
    ], style=styles['row']),

    # Third visualization row
    html.Div([
        html.Div([
            dcc.Graph(
                id='tests-scatter',
                figure=px.scatter(
                    df[df['Tests Per Million'] > 0],
                    x='Tests Per Million',
                    y='Cases Per Million',
                    color='Continent',
                    size='Population',
                    hover_name='Country',
                    title='Testing vs Case Rates',
                    labels={
                        'Tests Per Million': 'Tests per Million People',
                        'Cases Per Million': 'Cases per Million People'
                    }
                ).update_layout(
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)'
                )
            )
        ], style=styles['graph-container']),

        html.Div([
            dcc.Graph(
                id='top-countries',
                figure=px.bar(
                    df.nlargest(10, 'Total Cases'),
                    x='Country',
                    y='Total Cases',
                    color='Fatality Rate',
                    title='Top 10 Countries by Cases (Color by Fatality Rate)',
                    hover_data=['Total Deaths', 'Recovery Rate'],
                    color_continuous_scale='Viridis'
                ).update_layout(
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    yaxis_type="log"
                )
            )
        ], style=styles['graph-container'])
    ], style=styles['row']),

    # Data table
    html.Div([
        html.H3("Detailed Country Data", style={'textAlign': 'center'}),
        html.Div(id='data-table')
    ], style={'marginTop': '30px'}),

    # Footer
    html.Div([
        html.P("Data last updated: " + df['Last Updated'].iloc[0]),
        html.P("Data source: disease.sh API (John Hopkins University CSSE)")
    ], style=styles['footer'])
])

# Callbacks for interactive elements
@app.callback(
    Output('cases-map', 'figure'),
    [Input('metric-dropdown', 'value')]
)
def update_map(selected_metric):
    fig = px.choropleth(
        df,
        locations="ISO",
        color=selected_metric,
        hover_name="Country",
        hover_data=["Population", "Total Deaths", "Recovery Rate"],
        color_continuous_scale=px.colors.sequential.Plasma,
        title=f"Global COVID-19 {selected_metric}",
        projection="natural earth"
    )
    fig.update_geos(showcountries=True, showcoastlines=True, showland=True)
    fig.update_layout(
        margin={"r":0,"t":30,"l":0,"b":0},
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        geo=dict(bgcolor= 'rgba(0,0,0,0)')
    )
    return fig

@app.callback(
    Output('data-table', 'children'),
    [Input('cases-map', 'clickData')]
)
def update_table(clickData):
    if clickData is None:
        return html.Div([
            dash.dash_table.DataTable(
                data=df.sort_values('Total Cases', ascending=False).to_dict('records'),
                columns=[{"name": i, "id": i} for i in df.columns if i not in ['Latitude', 'Longitude', 'ISO']],
                page_size=10,
                style_table={'overflowX': 'auto', 'margin': '0 auto', 'width': '90%'},
                style_cell={
                    'height': 'auto',
                    'minWidth': '100px', 'width': '120px', 'maxWidth': '180px',
                    'whiteSpace': 'normal',
                    'textAlign': 'left'
                },
                style_header={
                    'backgroundColor': '#2c3e50',
                    'color': 'white',
                    'fontWeight': 'bold'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 248, 248)'
                    }
                ],
                filter_action="native",
                sort_action="native"
            )
        ])

    country = clickData['points'][0]['hovertext']
    country_data = df[df['Country'] == country]
    return html.Div([
        dash.dash_table.DataTable(
            data=country_data.to_dict('records'),
            columns=[{"name": i, "id": i} for i in country_data.columns if i not in ['Latitude', 'Longitude', 'ISO']],
            style_table={'overflowX': 'auto', 'margin': '0 auto', 'width': '90%'}
        )
    ])

# Run the app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8050) # Changed from app.run_server to app.run

<ipython-input-1-6f8ace542ece>:49: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Test Positivity Rate'] = (df['Total Cases'] / df['Total Tests'].replace(0, pd.NA)).fillna(0) * 100


<IPython.core.display.Javascript object>